In [7]:
import argparse
from PIL import Image
import cv2
import numpy as np
import os
import pandas as pd
import datetime
import tensorflow as tf
import sys
sys.path.append("C:/Users/AALY/object_detection/models/research/slim")
sys.path.append("C:/Users/AALY/object_detection/models/research/object_detection")
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util


def run_inference_for_single_image(image, sess, tensor_dict):
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict, feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


def load_model(PATH_TO_FROZEN_GRAPH: str, PATH_TO_LABELS: str):
    """
    Load model from frozen inference graph
    """
    # Load frozen inference graph into memory
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

    # Loading labelmap
    category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
    return graph, category_index


def run_surveillance(model_path, labelmap_path, show_video_steam, label_to_look_for, output_directory, threshold, label_to_look_for2):
    # Load model
    graph, category_index = load_model(model_path, labelmap_path)
    # Initialize Video Capture
    cap = cv2.VideoCapture("C:/Users/AALY/object_detection/zarbeazbtest.mp4")
    # Create output directory if not already created
    os.makedirs(output_directory, exist_ok=True)
    os.makedirs(output_directory+'/images', exist_ok=True)
    showcrime=0
    showuniform=0
    police_arrived = False 
    ToWriteOnlabel = ""
    if os.path.exists(output_directory+'/results.csv'):
        df = pd.read_csv(output_directory+'/results.csv')
    else:
        df = pd.DataFrame(columns=['timestamp', 'img_path'])

    # Open detection graph
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
            while True:
                try:
                    # Read frame from video
                    ret, image_np = cap.read()

                    # Copy image for later
                    image_show = np.copy(image_np)
#                     ToWriteOnlabel = ""
                    gunCount=0
                    armycount = 0
                    uniformCount = 0
                    image_height, image_width, _ = image_np.shape
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, sess, tensor_dict)

                    

                    # Get data(label, xmin, ymin, xmax, ymax)
                    output = []
                    for index, score in enumerate(output_dict['detection_scores']):
                            if score < threshold:
                                continue
                            label = category_index[output_dict['detection_classes'][index]]['name']
                            ymin, xmin, ymax, xmax = output_dict['detection_boxes'][index]
                            output.append((label, int(xmin * image_width), int(ymin * image_height), int(xmax * image_width), int(ymax * image_height)))

                    # Save incident (could be extended to send a email or something)
                    for l, x_min, y_min, x_max, y_max in output:

                        if l == label_to_look_for:
                            #CODE FOR SAVING IMAGES. 
#                             array = cv2.cvtColor(np.array(image_show), cv2.COLOR_RGB2BGR)
#                             image = Image.fromarray(array)
#                             cropped_img = image.crop((x_min, y_min, x_max, y_max))
#                             file_path = output_directory+'/images/'+str(len(df))+'.jpg'
#                             cropped_img.save(file_path, "JPEG", icc_profile=cropped_img.info.get('icc_profile'))
#                             df.loc[len(df)] = [datetime.datetime.now(), file_path]
#                             df.to_csv(output_directory+'/results.csv', index=None)
                            #ToWriteOnlabel = "Gun"
                            gunCount=gunCount+1
    
    
                    for l1, x_min1, y_min1, x_max1, y_max in output:
        
                        if l1 == label_to_look_for2:
                            #ToWriteOnlabel = "Uniform"
                            uniformCount = uniformCount+1
                                          
  
                    if show_video_steam:
                        # Visualize results
                        vis_util.visualize_boxes_and_labels_on_image_array(
                            image_np,
                            output_dict['detection_boxes'],
                            output_dict['detection_classes'],
                            output_dict['detection_scores'],
                            category_index,
                            instance_masks=output_dict.get('detection_masks'),
                            use_normalized_coordinates=True,
                            line_thickness=8)
                        if gunCount > uniformCount:
                            showcrime=showcrime+1
                            print(str(showcrime) + " While gun > uniform ")
                            
#                             if uniformCount > 2:
#                                 ToWriteOnlabel="POLICE ARRIVED"
#                                 showcrime = 0                            
                            
                            if showcrime > 40:
                                ToWriteOnlabel="CRIME DETECTED"
                                if showuniform > 5:
                                    showuniform = 0
                                #showuniform = 5 ##########################new addition########################

                            else:
                                ToWriteOnlabel=""
                                
#                         elif  gunCount == uniformCount and gunCount > 0:
#                             armycount = armycount + 1
                            
#                             if armycount > 10:
#                                 ToWriteOnlabel = "NOT A CRIME SCENE [POLICE/ARMY]"
                            
                        elif gunCount == uniformCount and gunCount > 0:
                            showuniform += 1
                            ToWriteOnlabel = "NOT A CRIME SCENE [POLICE/ARMY]"
                            print(str(showuniform) + " While gun == uniform")
                
                        elif uniformCount > gunCount:
                            showuniform = showuniform + 1
                            print(str(showuniform) + " While uniform > gun")
                            print("showuniform= "+str(showuniform))
                            if showuniform > 15:
                                ToWriteOnlabel = "NOT A CRIME SCENE [POLICE/ARMY]"
                                if showcrime > 20:
                                    showcrime = 15
                            else:
                                ToWriteOnlabel=""                                
                        
                        if ToWriteOnlabel == "CRIME DETECTED":
                            textColor = (0,0,255)
                        else:
                            textColor = (0,255,0)
                        
                        
                        font         = cv2.FONT_HERSHEY_SIMPLEX
                        bottomLeftCornerOfText = (150,30)
                        fontScale              = 1
                        fontColor              = textColor
                        lineType               = 2

                        cv2.putText(image_np,ToWriteOnlabel, 
                            bottomLeftCornerOfText, 
                            font, 
                            fontScale,
                            fontColor,
                            lineType) 
                        #=================TO DISPLAY GUN COUNT==============================
#                         font         = cv2.FONT_HERSHEY_SIMPLEX
#                         bottomLeftCornerOfText = (500,20)
#                         fontScale              = 1
#                         fontColor              = (255,255,255)
#                         lineType               = 2

#                         cv2.putText(image_np,"gun count = "+str(gunCount), 
#                             bottomLeftCornerOfText, 
#                             font, 
#                             fontScale,
#                             fontColor,
#                             lineType)   
#                         #==================TO DISPLAY UNIFORM COUNT===========================
#                         font         = cv2.FONT_HERSHEY_SIMPLEX
#                         bottomLeftCornerOfText = (500,50)
#                         fontScale              = 1
#                         fontColor              = (255,255,255)
#                         lineType               = 2

#                         cv2.putText(image_np,"uniform count = "+str(uniformCount), 
#                             bottomLeftCornerOfText, 
#                             font, 
#                             fontScale,
#                             fontColor,
#                             lineType)                         
                        
                        cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
                        if cv2.waitKey(25) & 0xFF == ord('q'):
                            cap.release()
                            cv2.destroyAllWindows()
                            break

                except Exception as e:
                    print(e)
                    cap.release()
                    cv2.destroyAllWindows()
                    break


# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='Surveillance System')
#     parser.add_argument('-m', '--model_path', type=str, required=True, help='Path to the frozen inference graph')
#     parser.add_argument('-l', '--labelmap', type=str, required=True, help='Path to labelmap')
#     parser.add_argument('-t', '--threshold', type=float, default=0.5, help='Threshold for bounding boxes')
#     parser.add_argument('-s', '--show', default=True, action='store_true', help='Show window')
#     parser.add_argument('-la', '--label', default='gun', type=str, help='Label name to detect')
#     parser.add_argument('-o', '--output_directory', default='results', type=str, help='Directory for the outputs')
#     args = parser.parse_args()
#     model1/frozen_inference_graphFinal/frozen_inference_graph_enhanced
run_surveillance("C:/Users/AALY/object_detection/frozengraph14-6-20.pb", 
                 "C:/Users/AALY/object_detection/label_mapFinal.pbtxt", True, 
                 "gun", "C:/Users/AALY/results", 0.5, "uniform")

1 While gun > uniform 
2 While gun > uniform 
3 While gun > uniform 
4 While gun > uniform 
5 While gun > uniform 
6 While gun > uniform 
7 While gun > uniform 
8 While gun > uniform 
9 While gun > uniform 
10 While gun > uniform 
11 While gun > uniform 
12 While gun > uniform 
13 While gun > uniform 
14 While gun > uniform 
15 While gun > uniform 
16 While gun > uniform 
17 While gun > uniform 
18 While gun > uniform 
19 While gun > uniform 
20 While gun > uniform 
21 While gun > uniform 
22 While gun > uniform 
23 While gun > uniform 
24 While gun > uniform 
25 While gun > uniform 
26 While gun > uniform 
27 While gun > uniform 
28 While gun > uniform 
29 While gun > uniform 
30 While gun > uniform 
31 While gun > uniform 
32 While gun > uniform 
33 While gun > uniform 
34 While gun > uniform 
35 While gun > uniform 
36 While gun > uniform 
37 While gun > uniform 
38 While gun > uniform 
39 While gun > uniform 
40 While gun > uniform 
41 While gun > uniform 
42 While gun > uniform 
4

201 While gun > uniform 
3 While gun == uniform
4 While uniform > gun
showuniform= 4
5 While uniform > gun
showuniform= 5
6 While uniform > gun
showuniform= 6
7 While uniform > gun
showuniform= 7
8 While uniform > gun
showuniform= 8
9 While uniform > gun
showuniform= 9
10 While uniform > gun
showuniform= 10
11 While gun == uniform
12 While uniform > gun
showuniform= 12
13 While uniform > gun
showuniform= 13
14 While uniform > gun
showuniform= 14
15 While uniform > gun
showuniform= 15
16 While uniform > gun
showuniform= 16
17 While gun == uniform
18 While gun == uniform
19 While uniform > gun
showuniform= 19
20 While uniform > gun
showuniform= 20
21 While uniform > gun
showuniform= 21
22 While uniform > gun
showuniform= 22
23 While uniform > gun
showuniform= 23
24 While uniform > gun
showuniform= 24
25 While uniform > gun
showuniform= 25
26 While uniform > gun
showuniform= 26
27 While uniform > gun
showuniform= 27
28 While uniform > gun
showuniform= 28
29 While uniform > gun
showuniform

82 While gun > uniform 
83 While gun > uniform 
84 While gun > uniform 
85 While gun > uniform 
2 While uniform > gun
showuniform= 2
3 While uniform > gun
showuniform= 3
4 While uniform > gun
showuniform= 4
5 While uniform > gun
showuniform= 5
6 While uniform > gun
showuniform= 6
7 While uniform > gun
showuniform= 7
8 While uniform > gun
showuniform= 8
9 While uniform > gun
showuniform= 9
10 While uniform > gun
showuniform= 10
11 While uniform > gun
showuniform= 11
12 While uniform > gun
showuniform= 12
13 While uniform > gun
showuniform= 13
14 While uniform > gun
showuniform= 14
15 While uniform > gun
showuniform= 15
16 While uniform > gun
showuniform= 16
17 While uniform > gun
showuniform= 17
18 While uniform > gun
showuniform= 18
19 While uniform > gun
showuniform= 19
20 While gun == uniform
21 While gun == uniform
22 While gun == uniform
23 While uniform > gun
showuniform= 23
24 While uniform > gun
showuniform= 24
25 While uniform > gun
showuniform= 25
26 While uniform > gun
showun

110 While gun > uniform 
111 While gun > uniform 
112 While gun > uniform 
113 While gun > uniform 
114 While gun > uniform 
115 While gun > uniform 
116 While gun > uniform 
117 While gun > uniform 
118 While gun > uniform 
119 While gun > uniform 
120 While gun > uniform 
121 While gun > uniform 
122 While gun > uniform 
123 While gun > uniform 
124 While gun > uniform 
125 While gun > uniform 
126 While gun > uniform 
127 While gun > uniform 
128 While gun > uniform 
129 While gun > uniform 
130 While gun > uniform 
131 While gun > uniform 
132 While gun > uniform 
133 While gun > uniform 
134 While gun > uniform 
135 While gun > uniform 
136 While gun > uniform 
137 While gun > uniform 
138 While gun > uniform 
139 While gun > uniform 
140 While gun > uniform 
141 While gun > uniform 
142 While gun > uniform 
143 While gun > uniform 
144 While gun > uniform 
145 While gun > uniform 
146 While gun > uniform 
147 While gun > uniform 
148 While gun > uniform 
149 While gun > uniform 


64 While uniform > gun
showuniform= 64
65 While uniform > gun
showuniform= 65
66 While uniform > gun
showuniform= 66
67 While uniform > gun
showuniform= 67
17 While gun > uniform 
68 While gun == uniform
69 While gun == uniform
70 While gun == uniform
18 While gun > uniform 
19 While gun > uniform 
71 While gun == uniform
72 While gun == uniform
20 While gun > uniform 
73 While uniform > gun
showuniform= 73
74 While uniform > gun
showuniform= 74
75 While uniform > gun
showuniform= 75
76 While uniform > gun
showuniform= 76
77 While uniform > gun
showuniform= 77
78 While uniform > gun
showuniform= 78
79 While uniform > gun
showuniform= 79
80 While uniform > gun
showuniform= 80
81 While uniform > gun
showuniform= 81
82 While uniform > gun
showuniform= 82
83 While uniform > gun
showuniform= 83
84 While uniform > gun
showuniform= 84
85 While uniform > gun
showuniform= 85
86 While gun == uniform
87 While uniform > gun
showuniform= 87
88 While gun == uniform
89 While uniform > gun
showuniform

...

58 While uniform > gun
showuniform= 58
59 While uniform > gun
showuniform= 59
60 While uniform > gun
showuniform= 60
61 While uniform > gun
showuniform= 61
16 While gun > uniform 
62 While uniform > gun
showuniform= 62
63 While uniform > gun
showuniform= 63
64 While uniform > gun
showuniform= 64
65 While uniform > gun
showuniform= 65
66 While uniform > gun
showuniform= 66
67 While uniform > gun
showuniform= 67
68 While uniform > gun
showuniform= 68
69 While gun == uniform
70 While uniform > gun
showuniform= 70
17 While gun > uniform 
18 While gun > uniform 
19 While gun > uniform 
71 While uniform > gun
showuniform= 71
72 While uniform > gun
showuniform= 72
73 While uniform > gun
showuniform= 73
74 While uniform > gun
showuniform= 74
75 While uniform > gun
showuniform= 75
76 While uniform > gun
showuniform= 76
77 While uniform > gun
showuniform= 77
78 While uniform > gun
showuniform= 78
79 While uniform > gun
showuniform= 79
80 While uniform > gun
showuniform= 80
81 While uniform > gun

8 While gun == uniform
9 While uniform > gun
showuniform= 9
10 While gun == uniform
11 While uniform > gun
showuniform= 11
12 While gun == uniform
13 While uniform > gun
showuniform= 13
14 While uniform > gun
showuniform= 14
15 While gun == uniform
16 While gun == uniform
17 While gun == uniform
18 While gun == uniform
19 While gun == uniform
20 While gun == uniform
21 While gun == uniform
22 While gun == uniform
23 While uniform > gun
showuniform= 23
24 While uniform > gun
showuniform= 24
25 While gun == uniform
16 While gun > uniform 
17 While gun > uniform 
26 While uniform > gun
showuniform= 26
27 While uniform > gun
showuniform= 27
28 While uniform > gun
showuniform= 28
29 While uniform > gun
showuniform= 29
30 While uniform > gun
showuniform= 30
31 While uniform > gun
showuniform= 31
32 While uniform > gun
showuniform= 32
33 While uniform > gun
showuniform= 33
34 While uniform > gun
showuniform= 34
35 While uniform > gun
showuniform= 35
36 While uniform > gun
showuniform= 36
37 W

101 While uniform > gun
showuniform= 101
102 While uniform > gun
showuniform= 102
103 While uniform > gun
showuniform= 103
104 While uniform > gun
showuniform= 104
105 While uniform > gun
showuniform= 105
106 While uniform > gun
showuniform= 106
107 While gun == uniform
108 While uniform > gun
showuniform= 108
109 While uniform > gun
showuniform= 109
110 While gun == uniform
111 While gun == uniform
112 While uniform > gun
showuniform= 112
113 While uniform > gun
showuniform= 113
114 While uniform > gun
showuniform= 114
115 While uniform > gun
showuniform= 115
116 While uniform > gun
showuniform= 116
117 While uniform > gun
showuniform= 117
20 While gun > uniform 
21 While gun > uniform 
118 While gun == uniform
22 While gun > uniform 
23 While gun > uniform 
119 While uniform > gun
showuniform= 119
120 While gun == uniform
16 While gun > uniform 
17 While gun > uniform 
18 While gun > uniform 
19 While gun > uniform 
20 While gun > uniform 
21 While gun > uniform 
22 While gun > unifo

29 While gun > uniform 
30 While gun > uniform 
31 While gun > uniform 
32 While gun > uniform 
33 While gun > uniform 
23 While gun == uniform
34 While gun > uniform 
35 While gun > uniform 
36 While gun > uniform 
37 While gun > uniform 
38 While gun > uniform 
39 While gun > uniform 
40 While gun > uniform 
41 While gun > uniform 
42 While gun > uniform 
43 While gun > uniform 
44 While gun > uniform 
45 While gun > uniform 
46 While gun > uniform 
47 While gun > uniform 
48 While gun > uniform 
49 While gun > uniform 
50 While gun > uniform 
51 While gun > uniform 
52 While gun > uniform 
53 While gun > uniform 
54 While gun > uniform 
55 While gun > uniform 
56 While gun > uniform 
57 While gun > uniform 
58 While gun > uniform 
59 While gun > uniform 
60 While gun > uniform 
61 While gun > uniform 
62 While gun > uniform 
63 While gun > uniform 
64 While gun > uniform 
65 While gun > uniform 
66 While gun > uniform 
67 While gun > uniform 
68 While gun > uniform 
69 While gun > u

111 While gun > uniform 
112 While gun > uniform 
113 While gun > uniform 
114 While gun > uniform 
115 While gun > uniform 
1 While gun == uniform
2 While gun == uniform
116 While gun > uniform 
3 While gun == uniform
4 While gun == uniform
5 While uniform > gun
showuniform= 5
6 While uniform > gun
showuniform= 6
117 While gun > uniform 
1 While uniform > gun
showuniform= 1
2 While uniform > gun
showuniform= 2
3 While uniform > gun
showuniform= 3
4 While uniform > gun
showuniform= 4
5 While uniform > gun
showuniform= 5
6 While uniform > gun
showuniform= 6
7 While uniform > gun
showuniform= 7
8 While uniform > gun
showuniform= 8
9 While uniform > gun
showuniform= 9
10 While uniform > gun
showuniform= 10
11 While uniform > gun
showuniform= 11
12 While uniform > gun
showuniform= 12
13 While uniform > gun
showuniform= 13
14 While uniform > gun
showuniform= 14
15 While uniform > gun
showuniform= 15
16 While uniform > gun
showuniform= 16
17 While uniform > gun
showuniform= 17
18 While unifo

In [2]:
a=4
a

4